In [1]:
import ee

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [73]:
import importlib
from flood_detection import modis
importlib.reload(modis)
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio as rio
import geopandas as gpd
from dateutil import rrule
from datetime import datetime
import rioxarray as rioxr

# specify these

I used a shapefile I found here: [Honduras Shapefile](https://data.humdata.org/dataset/cod-ab-hnd?)

Put it in `Data/Shapefiles/`, and unzip it

In [4]:
shapefilePath = Path('Data/Shapefiles/hnd_adm_sinit_20161005_SHP/hnd_admbnda_adm0_sinit_20161005.shp')

if not shapefilePath.exists():
    print("Shapefile not found!!!")
else:
    print("Shapefile found, good to go")

Shapefile found, good to go


### This is the output path where the rasters will go

In [5]:
outputPath = Path('Data/Rasters/GFD/') / shapefilePath.stem
outputPath.mkdir(exist_ok=True, parents=True)

### Start and end date of what you want to generate, with time resolution (dt)

In [30]:
startDate = '2020-01-01'
endDate = '2020-12-31'
dt = rrule.MONTHLY # either YEARLY, MONTHLY, or WEEKLY

# Prepare the data

In [13]:
# read shapefile
shapefile = gpd.read_file(shapefilePath)

In [16]:
# upload convex hull to gee (shapefile itself is usually too complex to be uploaded)
gdpFrame = gpd.GeoDataFrame(shapefile.convex_hull)
gdpFrame.columns = ['geom']
gdpFrame = gdpFrame.set_geometry('geom')
gdpFrame = gdpFrame.set_crs(shapefile.crs)
eeShape = geemap.geopandas_to_ee(gdpFrame).geometry()

In [47]:
# create vector of date to iterate over
dates = list(map(lambda date: date.strftime('%Y-%m-%d'), rrule.rrule(dt,
                      dtstart=datetime.strptime(startDate, '%Y-%m-%d'),
                      until=datetime.strptime(endDate, '%Y-%m-%d'))))

In [48]:
# specify delta time in ee format
ee_dt = 'Year'
if dt == rrule.MONTHLY:
    ee_dt = 'Month'
if dt == rrule.WEEKLY:
    ee_dt = 'Week'

# Download Data

In [66]:
def DownloadImageForDate(date, ee_dt, geom):
    
    # calculate start and end date
    startDate = ee.Date(date)
    endDate = ee.Date(date).advance(1, ee_dt)
    endDateString = endDate.format("YYYY-MM-dd").getInfo()

    # specify file name
    filename = outputPath/f"{date}_{endDateString}.tif"
    if filename.exists():
        return

    # create the floodmap code on ee
    flood_map = modis.dfo(geom, startDate, endDate, "standard", '3Day', True)
    flood_map_slope_mask = misc.apply_slope_mask(flood_map, thresh=5)
    perm_water = misc.get_jrc_perm(geom)
    dfo_final = ee.Image(flood_map_slope_mask).addBands(perm_water)
    
    # download
    geemap.download_ee_image(dfo_final, filename, crs='EPSG:4326', region=geom, scale=250)

In [67]:
# iterate over dates and dowload
for date in dates:
    DownloadImageForDate(date ee_dt, eeShape)

Collected and pre-processed MODIS Images
DFO Flood Dectection Complete


2020-01-01_2020-02-01.tif: |    | 0.00/106M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


# Postprocess the files

In [72]:
## add band descriptions to files

for file in outputPath.glob('**/*.tif'):
    with rio.open(file,'r+') as dst:
        dst.descriptions = tuple(['flooded', 'duration', 'clearViews', 'clearPerc', 'maxExtent', 'permanentWater'])

In [85]:
## Clip to geometry

for file in outputPath.glob('**/*.tif'):
    with rioxr.open_rasterio(file) as dst:
        data = dst
    data.rio.clip(shapefile.geometry).rio.to_raster(file)